In [29]:
import markdown
from IPython.display import display, Markdown
from agents import Agent, Runner, trace
import asyncio
from openai.types.responses import ResponseTextDeltaEvent

from dotenv import load_dotenv
import os
load_dotenv()
    
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
    
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
# disable tracing

OPENAI_AGENTS_DISABLE_TRACING=1
from agents import Agent, Runner, set_tracing_disabled
# Explicitly disable tracing
set_tracing_disabled(disabled=True)

OpenAI API Key set: Yes
Google API Key set: Yes


### Hello world example


Run a workflow starting at the given agent. The agent will run in a loop until a final output is generated. The loop runs like so: 1. The agent is invoked with the given input. 2. If there is a final output (i.e. the agent produces something of type agent.output_type, the loop terminates. 3. If there's a handoff, we run the loop again, with the new agent. 4. Else, we run tool calls (if any), and re-run the loop.


In two cases, the agent may raise an exception: 1. If the max_turns is exceeded, a MaxTurnsExceeded exception is raised. 2. If a guardrail tripwire is triggered, a GuardrailTripwireTriggered exception is raised.

Note that only the first agent's input guardrails are run.


```
run(
    starting_agent: Agent[TContext],
    input: str | list[TResponseInputItem],
    *,
    context: TContext | None = None,
    max_turns: int = DEFAULT_MAX_TURNS,
    hooks: RunHooks[TContext] | None = None,
    run_config: RunConfig | None = None,
    previous_response_id: str | None = None,
) -> RunResult
```


You can run agents via the Runner class. You have 3 options:

    Runner.run(), which runs async and returns a RunResult.
    Runner.run_sync(), which is a sync method and just runs .run() under the hood.
    Runner.run_streamed(), which runs async and returns a RunResultStreaming. It calls the LLM in streaming mode, and streams those events to you as they are received.


In [3]:

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

#result = Runner.run_sync(agent, "Write a haiku about recursion in programming.")
#print(result.final_output)

async def main():
    result = await Runner.run(agent, "What is the capital of France?")
    print(result.final_output)
    

if __name__ == "__main__":
    #asyncio.run(main())
    await main()

# Code within the code,
# Functions calling themselves,
# Infinite loop's dance.

The capital of France is Paris.


In [4]:
async def main():
    agent = Agent(name="Assistant", instructions="Reply very concisely.")
    thread_id = 1234

    with trace(workflow_name="Conversation", group_id=thread_id):
        # First turn
        result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
        print(result.final_output)
        # San Francisco

        # Second turn
        new_input = result.to_input_list() + [{"role": "user", "content": "What state is it in?"}]
        result = await Runner.run(agent, new_input)
        print(result.final_output)
        # California
if __name__ == "__main__":
    #asyncio.run(main())
    await main()
    
#result = await Runner.run(agent, "What is the weather in Los Angeles?")
#print(result.final_output)

San Francisco.
California.


#### First agent

- https://github.com/openai/openai-agents-python/tree/main/examples
- https://openai.github.io/openai-agents-python/

In [5]:

agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model="gpt-4o"
)


result = Runner.run_streamed(agent, input="what is square root of 4.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)
        



The square root of a number is a value that, when multiplied by itself, gives the original number. For the number 4, the square root is found by determining what number multiplied by itself equals 4.

\[ \sqrt{4} = x \]

This translates to:

\[ x \times x = 4 \]

If you try:

- \( x = 2 \), then \( 2 \times 2 = 4 \)
- \( x = -2 \), then \( (-2) \times (-2) = 4 \)

So, the square roots of 4 are 2 and -2. However, the principal (or non-negative) square root is typically what is referred to, which is 2.

Thus, the square root of 4 is 2.

In [6]:
result

RunResultStreaming(input='what is square root of 4.', new_items=[MessageOutputItem(agent=Agent(name='Math Tutor', instructions='You provide help with math problems. Explain your reasoning at each step and include examples', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_6834dcf73cac81918c6bcc8d47ffc8d80daf58e12cbe62f7', content=[ResponseOutputText(annotations=[], text='The square root of a number is a value that, when multiplied by itself, gives the original number. For t

In [7]:
async def main():
    #agent = Agent(name="Assistant", instructions="You are a helpful assistant")
    agent = Agent(
       name="Math Tutor",
       instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
       model="gpt-4o"
    )
    result = await Runner.run(agent, "what is square root of 4")
    print(result.final_output)

if __name__ == "__main__":
    #asyncio.run(main())
    await main()

The square root of a number is a value that, when multiplied by itself, gives the original number. For the square root of 4:

1. We need to find a number that, when squared (multiplied by itself), equals 4.
2. The numbers that satisfy this are 2 and -2 because:
   - \(2 \times 2 = 4\)
   - \((-2) \times (-2) = 4\) (since a negative times a negative equals a positive)

Therefore, the square roots of 4 are \(2\) and \(-2\). 

In the context of the principal square root, which typically refers to the non-negative root, the answer is \(2\).


In [8]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

result = Runner.run_streamed(
    agent,
    input="What is square root of 4.",
)
print("=== Run starting ===")

async for event in result.stream_events():
    # Ignore the raw responses 
    if event.type == "raw_response_event":
        continue
    # print agent updates
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # print generated items
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

=== Run starting ===
Agent updated: Math Tutor
-- Message output:
 The square root of a number is a value that, when multiplied by itself, gives the original number. Let's find the square root of 4.

1. **Understanding Square Roots:**
   - The notation for square root is the radical symbol \( \sqrt{} \).
   - So, when we say \( \sqrt{4} \), we're looking for a number \( x \) such that \( x \times x = 4 \).

2. **Calculate:**
   - We determine \( x \) by considering numbers that multiply to 4.
   - Both \( 2 \times 2 = 4 \) and \( -2 \times -2 = 4 \).

3. **Conclude:**
   - Therefore, the square roots of 4 are 2 and -2.

4. **Final Answer:**
   - \( \sqrt{4} = 2 \) (by convention, we usually refer to the principal, or positive, root).

**Example:**
- For another example, \( \sqrt{9} = 3 \) because \( 3 \times 3 = 9 \). Similarly, \( -3 \times -3 = 9 \), so the roots are 3 and -3. However, the principal root is 3.
=== Run complete ===


In [9]:
for k, v in result.__dict__.items():
  print(f"Key = {k}: Value = {v}")

Key = input: Value = What is square root of 4.
Key = new_items: Value = [MessageOutputItem(agent=Agent(name='Math Tutor', instructions='You provide help with math problems. Explain your reasoning at each step and include examples', handoff_description=None, handoffs=[], model='gpt-4o', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='msg_6834dcfed574819188669f3dbea0f51501b4fde3dbad151d', content=[ResponseOutputText(annotations=[], text="The square root of a number is a value that, when multiplied by itself, gives the original numbe

In [10]:
from agents import ModelSettings


advanced_agent = Agent(
   name="Advanced Assistant",
   instructions="""You are a an advnanced assistant who knows about engineering facts. Give 
   accurate information. If you don't know something, clearly state that.
   Take a pass if you don't know. """,
   model="gpt-4o",
   model_settings=ModelSettings(
       temperature=0.3,  # Lower for more deterministic outputs (0.0-2.0)
       max_tokens=1024,  # Maximum length of response
   ),
   tools=[]  # We'll cover tools in a later section
)
result = await Runner.run(agent, "Tell me about quantum computers.")
print(result.final_output)


Quantum computers are a type of computation system that leverage the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (either a 0 or a 1), quantum computers use quantum bits, or qubits.

### Key Concepts:

1. **Qubits:**
   - Qubits can exist in a state of 0, 1, or any combination of both simultaneously, thanks to the quantum property known as superposition. This allows quantum computers to perform many calculations simultaneously.

2. **Superposition:**
   - Superposition allows qubits to process multiple possibilities at once. For example, with two classical bits, there can be only one of the four states (00, 01, 10, or 11) at any time. But two qubits can represent all four states simultaneously.

3. **Entanglement:**
   - This is a phenomenon where qubits become intertwined and the state of one qubit can depend on the state of another, no matter the distance apart. This property is used to link qubits in 

### Streaming Example

In [11]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

Sure, here they are:

1. Why don't skeletons fight each other?
   Because they don't have the guts!

2. What do you call fake spaghetti?
   An impasta!

3. Why did the scarecrow win an award?
   Because he was outstanding in his field!

4. How does the ocean say hello?
   It waves!

5. Why don't eggs tell jokes?
   Because they might crack up!

### Handoff Examples

In [12]:
from agents import Agent, Runner
import asyncio

french_agent = Agent(
    name="french agent",
    instructions="You only speak French.",
)

italian_agent = Agent(
    name="italian agent",
    instructions="You only speak Italian.",
)

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent, french_agent, italian_agent],
)


async def main():
    input="Hola, ¿cómo estás?"
    result = await Runner.run(triage_agent, input=input)
    print(f" Input = {input}, Output = {result.final_output}")
    input="Hello How are you?"
    result = await Runner.run(triage_agent, input=input)
    print(f" Input = {input}, Output = {result.final_output}")
    input="bonjour comment allez-vous?"
    result = await Runner.run(triage_agent, input=input )
    print(f" Input = {input}, Output = {result.final_output}")
    input="Salve, come stai?"
    result = await Runner.run(triage_agent, input=input)
    print(f" Input = {input}, Output = {result.final_output}")
    #print(result.final_output)
    # ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

 Input = Hola, ¿cómo estás?, Output = ¡Hola! Estoy bien, gracias. ¿Y tú?
 Input = Hello How are you?, Output = Hello! I'm doing well, thank you. How about you?
 Input = bonjour comment allez-vous?, Output = Bonjour ! Je vais bien, merci. Et vous, comment allez-vous ?
 Input = Salve, come stai?, Output = Ciao! Sto bene, grazie. E tu, come stai?


In [13]:
from agents import Agent

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [14]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

In [15]:
async def main():
    result = await Runner.run(triage_agent, input="Where is 2028 olympics?")
    print(result.final_output)
    result = await Runner.run(triage_agent, input="What is 0 + 1?")
    print(result.final_output)


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

The 2028 Summer Olympics will be held in Los Angeles, California, USA. This will mark the third time Los Angeles has hosted the Games, having previously done so in 1932 and 1984.
The problem \(0 + 1\) asks us to add zero to one.

1. **Understanding Zero:** Zero added to any number does not change the value of that number. This is known as the additive identity property of zero.

2. **Performing the Addition:** 
   - Start with 1.
   - Add 0 to it: \(1 + 0 = 1\).

So, \(0 + 1 = 1\).

**Example:**

- If you have 1 apple and add 0 more apples, you still have 1 apple. 

Therefore, adding zero to a number leaves you with the original number.


### Run the agent orchestration

In [17]:
from agents import Runner

async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")
    print(result.final_output)

if __name__ == "__main__":
    #asyncio.run(main())
    await main()

The capital of France is Paris. Paris has been a significant cultural, political, and economic center throughout history. It's known for landmarks like the Eiffel Tower, the Louvre Museum, and its influence on art, fashion, and philosophy.


### Function Example

In [18]:
import asyncio

from agents import Agent, Runner, function_tool


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

The weather in Tokyo is sunny.


### Add a guardrail

In [19]:
from agents import GuardrailFunctionOutput, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [26]:
async def main():
    input="who was the first president of the united states?"
    result = await Runner.run(guardrail_agent, input)
    print(f" Input = {input}, Output = {result.final_output}")
    input = "what is 1 + 1"
    result = await Runner.run(triage_agent, input )
    print(f" Input = {input}, Output = {result.final_output}")
    input = "what is life?"
    result = await Runner.run(triage_agent, input )
    print(f" Input = {input}, Output = {result.final_output}")
    input = "who was the first president of University of California system?"
    result = await Runner.run(guardrail_agent, input)
    print(f" Input = {input}, Output = {result.final_output}")
    

if __name__ == "__main__":
    #asyncio.run(main())
    await main()

 Input = who was the first president of the united states?, Output = is_homework=True reasoning='The question pertains to a common historical fact often found in educational settings, specifically in history classes or assignments.'
 Input = what is 1 + 1, Output = The problem you're asking about is a simple addition, specifically:

1 + 1

### Step-by-step Explanation:

1. **Understanding Addition**: Addition is a basic arithmetic operation that combines two or more numbers to find their total. In this case, we are adding the number 1 to another number 1.

2. **Line Up the Numbers**: Although it's straightforward in this case, for larger numbers, you might line them up vertically to add.

   \[
   \begin{array}{c}
   \ 1 \\
+ \ 1 \\
\hline
   \ ?
   \end{array}
   \]

3. **Add the Numbers**: When we add 1 and 1, we simply increase the first 1 by 1 more.

4. **Finding the Total**: The sum of 1 and 1 is 2.

### Conclusion
So, the answer to 1 + 1 is 2.

### Additional Example

- **2 + 3**

### Put it all together

In [37]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)


async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    print(f"Guardrail check is {final_output} \n")
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)

async def main():
    result = await Runner.run(triage_agent, "who was the first president of the united states?")
    print(result.final_output)


    # #result = await Runner.run(triage_agent, "what is life")
    # result = await Runner.run(triage_agent, "who was the first president of University of California system?")
    # print(result.final_output)
    # #print(result.final_output)
    result = await Runner.run(triage_agent, "what is 1 + 1")
    print(result.final_output)


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

Guardrail check is is_homework=True reasoning='The question is a straightforward historical fact that is commonly covered in educational settings, specifically about a prominent historical figure. This type of question is often found in school assignments.' 

The first president of the United States was George Washington. He served from 1789 to 1797 after being unanimously elected by the Electoral College. Washington is often called the "Father of His Country" for his leadership during the American Revolutionary War and his role in shaping the early federal government. His presidency set many precedents, including the tradition of a two-term limit, which was later formalized by the 22nd Amendment.
Guardrail check is is_homework=True reasoning='The user is asking a basic arithmetic question, which is often associated with homework or learning practice.' 

The problem you're asking is a simple addition problem: \(1 + 1\).

### Explanation:
- **Addition** is a basic arithmetic operation w

In [38]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent( 
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail( 
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output, 
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(  
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
)

async def main():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except InputGuardrailTripwireTriggered:
        print("Math homework guardrail tripped")
await main()

Math homework guardrail tripped


### output guardrails

In [39]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)
class MessageOutput(BaseModel): 
    response: str

class MathOutput(BaseModel): 
    reasoning: str
    is_math: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
)

@output_guardrail
async def math_guardrail(  
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, output.response, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

agent = Agent( 
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail],
    output_type=MessageOutput,
)

async def main():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except OutputGuardrailTripwireTriggered:
        print("Math output guardrail tripped")
await main()

Math output guardrail tripped


### event types for streamed events

In [40]:
import logging

logger = logging.getLogger("openai.agents") # or openai.agents.tracing for the Tracing logger

# To make all logs show up
logger.setLevel(logging.DEBUG)
# To make info and above show up
logger.setLevel(logging.INFO)
# To make warning and above show up
logger.setLevel(logging.WARNING)
# etc

# You can customize this as needed, but this will output to `stderr` by default
logger.addHandler(logging.StreamHandler())

In [41]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    #return random.randint(1, 10)
    no_str = input("Enter number of jokes: ")
    number = int(no_str)  # Convert the input string to an integer
    print(f"You asked for {number} jokes")
    return number


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


if __name__ == "__main__":
    #asyncio.run(main())
    await main()

=== Run starting ===
Agent updated: Joker


Enter number of jokes:  2


You asked for 2 jokes
-- Tool was called
-- Tool output: 2
-- Message output:
 Alright, here are two jokes for you:

1. **Why don't scientists trust atoms?**  
   Because they make up everything!

2. **What do you get when you cross a snowman and a vampire?**  
   Frostbite!
=== Run complete ===


### Agents as tools and orchestration

In [42]:
from agents import Agent, Runner
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

async def main():
    result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
    print(result.final_output)

await main()

"Hello, how are you?" in Spanish is "Hola, ¿cómo estás?"


#### Ask your translation questions in any language including websearch

In [34]:
import asyncio
from agents import Agent, FileSearchTool, Runner, WebSearchTool
from agents import Agent, ItemHelpers, MessageOutputItem, Runner, trace

"""
This example shows the agents-as-tools pattern. The frontline agent receives a user message and
then picks which agents to call, as tools. In this case, it picks from a set of translation
agents.
"""

malaylam_agent = Agent(
    name="malaylam_agent",
    instructions="You translate the user's message to Malaylam",
    handoff_description="An english to Malaylam translator",
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You translate the user's message to Spanish",
    handoff_description="An english to spanish translator",
)

french_agent = Agent(
    name="french_agent",
    instructions="You translate the user's message to French",
    handoff_description="An english to french translator",
)

italian_agent = Agent(
    name="italian_agent",
    instructions="You translate the user's message to Italian",
    handoff_description="An english to italian translator",
)
websearch_agent = Agent(
    name="websearch_agent",
    instructions=f"You are a helpful agent who can search web, respond to questions by using the search tool",
    model="gpt-4o",
    tools=[WebSearchTool()]
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools in order."
        "You never translate on your own, you always use the provided tools."
    ),
    tools=[
        malaylam_agent.as_tool(
            tool_name="translate_to_malaylam",
            tool_description="Translate the user's message to Malayalam",
        ),
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
        italian_agent.as_tool(
            tool_name="translate_to_italian",
            tool_description="Translate the user's message to Italian",
        ),
        websearch_agent.as_tool(
            tool_name="websearch_agent",
            tool_description="search web to find answers to user query",
        ),

    ],
)

synthesizer_agent = Agent(
    name="synthesizer_agent",
    instructions="You inspect translations, correct them if needed, and produce a final concatenated response.",
)


async def main():
    msg = input("Hi! What would you like translated, and to which languages? ")

    # Run the entire orchestration in a single trace
    with trace("Orchestrator evaluator"):
        orchestrator_result = await Runner.run(orchestrator_agent, msg, max_turns=5,)
        for item in orchestrator_result.new_items:
            if hasattr(item, 'agent'):
               print(f" Agent: {item.agent.name}")
            if isinstance(item, MessageOutputItem):
                text = ItemHelpers.text_message_output(item)
                if text:
                    print(f"  - Translation step: {text}")

        synthesizer_result = await Runner.run(
            synthesizer_agent, orchestrator_result.to_input_list()
        )

    print(f"\n\nFinal response:\n{synthesizer_result.final_output}")
    

# async def main():
#     result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Spanish.")
#     print(result.final_output)

if __name__ == "__main__":
    #asyncio.run(main())
    await main()

Hi! What would you like translated, and to which languages?  Say How are you in Malayalam


 Agent: orchestrator_agent
 Agent: orchestrator_agent
 Agent: orchestrator_agent
  - Translation step: "How are you" in Malayalam is "സുഖമാണോ?"


Final response:
"How are you" in Malayalam is "സുഖമാണോ?"


### Mixing models

In [52]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
    model="o3-mini", 
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
    model=OpenAIChatCompletionsModel( 
        model="gpt-4o",
        openai_client=AsyncOpenAI()
    ),
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
    #handoffs=[english_agent],
    model="gpt-3.5-turbo",
)

async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)

await main()

¡Hola! Estoy muy bien, gracias. ¿Y tú, cómo te encuentras?


### Other vendor agents

In [44]:
#claude_agent = Agent(model="litellm/anthropic/claude-3-5-sonnet-20240620", ...)
#gemini_agent = Agent(model="litellm/gemini/gemini-2.5-flash-preview-04-17", ...)

In [2]:
from litellm import completion
import os
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
os.environ['GEMINI_API_KEY'] = GOOGLE_API_KEY
response = completion(
    model="gemini/gemini-2.0-flash", 
    #messages=[{"role": "user", "content": "write code for saying hi from LiteLLM"}]
    messages=[{"role": "user", "content": "who wrote first hello world?"}]
)
response.choices[0].message


Message(content='Brian Kernighan is credited with writing the first known "Hello, World!" program.  He used it in his 1972 Bell Laboratories internal memorandum, "Programming in C: A Tutorial."  While not the absolute first program ever, it became widely popularized through his and Dennis Ritchie\'s influential book, "The C Programming Language."\n', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None)

### Websearch Tool

In [46]:
from agents import Agent, FileSearchTool, Runner, WebSearchTool

In [31]:
websearch_agent = Agent(name="websearch_agent Assistant",
              instructions=f"You are a helpful agent, respond to questions by using the search tool",
              model="gpt-4o",
              tools=[WebSearchTool()]
              )

In [32]:
result = await Runner.run(websearch_agent, "who is the chancellor of university of california, Los Angeles?")

print(result.final_output)

As of May 26, 2025, Dr. Julio Frenk serves as the Chancellor of the University of California, Los Angeles (UCLA). He assumed this role on January 1, 2025, becoming the university's seventh chancellor and its first Latino leader. ([chancellor.ucla.edu](https://chancellor.ucla.edu/about/chancellor?utm_source=openai))

Prior to joining UCLA, Dr. Frenk was the president of the University of Miami from 2015 to 2024. He also served as the dean of the Harvard T.H. Chan School of Public Health and as Mexico's Secretary of Health, where he implemented a universal health insurance program benefiting over 55 million uninsured individuals. ([universityofcalifornia.edu](https://www.universityofcalifornia.edu/news/julio-frenk-take-helm-ucla-nations-no-1-ranked-public-university-its-7th-chancellor?utm_source=openai))

Dr. Frenk's leadership is guided by the principle of "reciprocating the generosity of strangers," a value inspired by his father's experience as a Jewish refugee in 1930s Mexico. He emp

In [51]:
result.new_items[1].raw_item.content[0].annotations

[AnnotationURLCitation(end_index=316, start_index=229, title='Julio Frenk - UCLA Office of the Chancellor', type='url_citation', url='https://chancellor.ucla.edu/about/chancellor?utm_source=openai'),
 AnnotationURLCitation(end_index=633, start_index=546, title='Julio Frenk - UCLA Office of the Chancellor', type='url_citation', url='https://chancellor.ucla.edu/about/chancellor?utm_source=openai'),
 AnnotationURLCitation(end_index=968, start_index=881, title='Julio Frenk - UCLA Office of the Chancellor', type='url_citation', url='https://chancellor.ucla.edu/about/chancellor?utm_source=openai'),
 AnnotationURLCitation(end_index=1191, start_index=1022, title='UCLA names new chancellor as campus is still reeling from protests over Israel-Hamas war', type='url_citation', url='https://apnews.com/article/a7af046f6f3383e0266d17ff9f32e465?utm_source=openai'),
 AnnotationURLCitation(end_index=1325, start_index=1194, title='Julio Frenk Is Bringing His Vision of Inclusivity to UCLA', type='url_cita